<a href="https://colab.research.google.com/github/EShoukry/AI-StoryGeneration/blob/master/DOC2VEC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing

In [ ]:
from __future__ import print_function
#import Keras library
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM, Input, Bidirectional
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.metrics import categorical_accuracy
!pip install spacy
#import spacy, and spacy french model
# spacy is used to work on text
import spacy.cli
spacy.cli.download("en_core_web_sm")
nlp = spacy.load('en_core_web_sm')

#import other libraries
import numpy as np
import random
import sys
import os
import time
import codecs
import collections
from six.moves import cPickle


In [ ]:
data_directory = ''
save_directory = ''
rnn_size = 128
batch_size = 30
seq_length = 15
num_epochs = 8
learning_rate = 0.001
input_file = ["test.txt"]
vocab_file = os.path.join(save_directory, "words_vocab.pkl")
sequences_step = 1

# Method to Create Wordlist

In [ ]:
def create_wordlist(doc):
    word_list = []
    for word in doc:
        if word.text not in ("\n", "\n\n", "Armor", "Class", "Speed"):
            word_list.append(word.text.lower())
    return word_list

# Build Wordlist

In [ ]:
list_of_words = []

input_file = 'test.txt'
with codecs.open(input_file, "r", encoding="utf-8") as f:
    data = f.read()
x_text = data.split()
doc = nlp(data)
w = create_wordlist(doc)
list_of_words = list_of_words + w

# Create Dictionary

In [ ]:
word_counts = collections.Counter(list_of_words)

# Mapping from index to word : that's the vocabulary
vocabulary_inv = [x[0] for x in word_counts.most_common()]
vocabulary_inv = list(sorted(vocabulary_inv))

# Mapping from word to index
vocab = {x: i for i, x in enumerate(vocabulary_inv)}
words = [x[0] for x in word_counts.most_common()]

#size of the vocabulary
vocab_size = len(words)
print("vocab size: ", vocab_size)

#save the words and vocabulary
with open(os.path.join(vocab_file), 'wb') as f:
    cPickle.dump((words, vocab, vocabulary_inv), f)

vocab size:  6929


In [ ]:
#create sequences
sequences = []
next_words = []
seq_length = 15
for i in range(0, len(list_of_words) - seq_length, sequences_step):
    sequences.append(list_of_words[i: i + seq_length])
    next_words.append(list_of_words[i + seq_length])

print('nb sequences:', len(sequences))

nb sequences: 118020


In [ ]:
X = np.zeros((len(sequences), seq_length, vocab_size), dtype=np.bool)
y = np.zeros((len(sequences), vocab_size), dtype=np.bool)
for i, sentence in enumerate(sequences):
    for t, word in enumerate(sentence):
        X[i, t, vocab[word]] = 1
    y[i, vocab[next_words[i]]] = 1